### Deliverable 3: Optimize the Model

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN','NAME'],axis=1,inplace=True)
application_df.head(5)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine which values to replace if counts are less than ...?
replace_application = [i for i in application_df['APPLICATION_TYPE'].value_counts().index if
     application_df['APPLICATION_TYPE'].value_counts()[i]<500]

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ..?
replace_class = [application_df['CLASSIFICATION'].value_counts().index[i] for i in
                 range(0,len(application_df['CLASSIFICATION'].value_counts())) if
     application_df['CLASSIFICATION'].value_counts().values[i]<1000]

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [6]:
# Generate our categorical variable lists
#application_df.dtypes

application_cat = ['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION',
                   'INCOME_AMT','SPECIAL_CONSIDERATIONS']

In [7]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\Alexis\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
# Merge one-hot encoded features and drop the originals
application_enc_df = pd.concat([application_df,encode_df],axis=1).reindex(application_df.index)
application_enc_df.drop(columns=application_cat,axis=1,inplace=True)
application_enc_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Split our preprocessed data into our features and target arrays
y = application_enc_df['IS_SUCCESSFUL'].values
X = application_enc_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

C:\Users\Alexis\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 3: Optimize Model 1st Attempt: More Nodes

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])
nodes_layer1 = 120
nodes_layer2 = 40

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_layer1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               5280      
                                                                 
 dense_1 (Dense)             (None, 40)                4840      
                                                                 
 dense_2 (Dense)             (None, 1)                 41        
                                                                 
Total params: 10,161
Trainable params: 10,161
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints2/",exist_ok=True)
checkpoint_path = "checkpoints2/weights.{epoch:02d}.hdf5"


# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True, save_freq=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5710 - accuracy: 0.7203
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5562 - accuracy: 0.7294
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5533 - accuracy: 0.7302
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5511 - accuracy: 0.7317
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5497 - accuracy: 0.7316
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5487 - accuracy: 0.7313
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5482 - accuracy: 0.7311
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5472 - accuracy: 0.7330
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5463 - accuracy: 0.7325
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5461 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5352 - accuracy: 0.7395
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5353 - accuracy: 0.7391
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5351 - accuracy: 0.7400
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5353 - accuracy: 0.7386
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5350 - accuracy: 0.7398
Epoch 86/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5351 - accuracy: 0.7399
Epoch 87/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5348 - accuracy: 0.7398
Epoch 88/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5349 - accuracy: 0.7395
Epoch 89/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5349 - accuracy: 0.7394
Epoch 90/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5348 - accuracy: 

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5628 - accuracy: 0.7299 - 393ms/epoch - 1ms/step
Loss: 0.5628162026405334, Accuracy: 0.729912519454956


# Deliverable 3: Optimize Model 2st Attempt: Change Bins¶

In [16]:
application_df2 = pd.read_csv("../charity_data.csv")
application_df2.drop(columns=['EIN','NAME'],axis=1,inplace=True)

replace_application = [i for i in application_df2['APPLICATION_TYPE'].value_counts().index if
     application_df2['APPLICATION_TYPE'].value_counts()[i]<1000]

# Replace in dataframe
for app in replace_application:
    application_df2.APPLICATION_TYPE = application_df2.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df2.APPLICATION_TYPE.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [17]:
# Determine which values to replace if counts are less than ..?
replace_class = [application_df2['CLASSIFICATION'].value_counts().index[i] for i in
                 range(0,len(application_df2['CLASSIFICATION'].value_counts())) if
     application_df2['CLASSIFICATION'].value_counts().values[i]<5000]

# Replace in dataframe
for cls in replace_class:
    application_df2.CLASSIFICATION = application_df2.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df2.CLASSIFICATION.value_counts()

C1000    17326
Other    10899
C2000     6074
Name: CLASSIFICATION, dtype: int64

In [18]:
# Generate our categorical variable lists
application_cat = ['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION',
                   'INCOME_AMT','SPECIAL_CONSIDERATIONS']

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df2[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

application_enc_df2 = pd.concat([application_df2,encode_df],axis=1).reindex(application_df2.index)
application_enc_df2.drop(columns=application_cat,axis=1,inplace=True)
application_enc_df2.head()

C:\Users\Alexis\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
# Split our preprocessed data into our features and target arrays
y = application_enc_df2['IS_SUCCESSFUL'].values
X = application_enc_df2.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

C:\Users\Alexis\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
nodes_layer1 = 120
nodes_layer2 = 40

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=nodes_layer1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=nodes_layer2, activation='relu'))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 120)               4560      
                                                                 
 dense_4 (Dense)             (None, 40)                4840      
                                                                 
 dense_5 (Dense)             (None, 1)                 41        
                                                                 
Total params: 9,441
Trainable params: 9,441
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compile the model
nn2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints3/",exist_ok=True)
checkpoint_path = "checkpoints3/weights.{epoch:02d}.hdf5"


# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True, save_freq=5)

# Train the model
fit_model = nn2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5797 - accuracy: 0.7178
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5659 - accuracy: 0.7244
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5633 - accuracy: 0.7265
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5610 - accuracy: 0.7279
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5604 - accuracy: 0.7283
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5593 - accuracy: 0.7290
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5586 - accuracy: 0.7297
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5576 - accuracy: 0.7293
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5572 - accuracy: 0.7290
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5568 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5479 - accuracy: 0.7349
Epoch 82/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5485 - accuracy: 0.7351
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5474 - accuracy: 0.7365
Epoch 84/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7351
Epoch 85/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5478 - accuracy: 0.7351
Epoch 86/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5474 - accuracy: 0.7353
Epoch 87/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5479 - accuracy: 0.7355
Epoch 88/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5479 - accuracy: 0.7349
Epoch 89/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5479 - accuracy: 0.7357
Epoch 90/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5475 - accuracy: 

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5686 - accuracy: 0.7255 - 347ms/epoch - 1ms/step
Loss: 0.5686041712760925, Accuracy: 0.7254810333251953


# Deliverable 3: Optimize Model 3rd Attempt: Activation Functions¶¶

In [35]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
nodes_layer1 = 120
nodes_layer2 = 40

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=nodes_layer1, input_dim=input_features, activation='tanh'))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=nodes_layer2, activation='tanh'))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 120)               4560      
                                                                 
 dense_7 (Dense)             (None, 40)                4840      
                                                                 
 dense_8 (Dense)             (None, 1)                 41        
                                                                 
Total params: 9,441
Trainable params: 9,441
Non-trainable params: 0
_________________________________________________________________


In [36]:
# Compile the model
nn3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints4/",exist_ok=True)
checkpoint_path = "checkpoints4/weights.{epoch:02d}.hdf5"


# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True, save_freq=5)

# Train the model
fit_model = nn3.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5788 - accuracy: 0.7203
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5672 - accuracy: 0.7256
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5634 - accuracy: 0.7255
Epoch 4/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5611 - accuracy: 0.7275
Epoch 5/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5597 - accuracy: 0.7278
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5592 - accuracy: 0.7296
Epoch 7/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5581 - accuracy: 0.7298
Epoch 8/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5578 - accuracy: 0.7278
Epoch 9/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5568 - accuracy: 0.7298
Epoch 10/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5563 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5449 - accuracy: 0.7372
Epoch 82/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5448 - accuracy: 0.7368
Epoch 83/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5448 - accuracy: 0.7371
Epoch 84/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5446 - accuracy: 0.7365
Epoch 85/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5451 - accuracy: 0.7368
Epoch 86/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5448 - accuracy: 0.7364
Epoch 87/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5449 - accuracy: 0.7370
Epoch 88/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5447 - accuracy: 0.7365
Epoch 89/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5444 - accuracy: 0.7365
Epoch 90/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5446 - accuracy: 

804/804 [==============================] - 1s 2ms/step - loss: 0.5432 - accuracy: 0.7375
Epoch 161/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5432 - accuracy: 0.7378
Epoch 162/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5433 - accuracy: 0.7382
Epoch 163/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5436 - accuracy: 0.7373
Epoch 164/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5432 - accuracy: 0.7370
Epoch 165/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5430 - accuracy: 0.7379
Epoch 166/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5432 - accuracy: 0.7382
Epoch 167/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5436 - accuracy: 0.7377
Epoch 168/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5433 - accuracy: 0.7374
Epoch 169/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5433 - a

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5726 - accuracy: 0.7241 - 350ms/epoch - 1ms/step
Loss: 0.5725765824317932, Accuracy: 0.7240816354751587
